<a href="https://colab.research.google.com/github/hmg8906/CienciaDatosCN/blob/main/Cob_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#inicializar y autenticar GEE
import ee
ee.Authenticate()
ee.Initialize(project="hmgarcia56ee1")

In [ ]:
import geemap
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import joblib


# 1. Cargar ROI (Huila)
departamentos = ee.FeatureCollection("FAO/GAUL/2015/level1")
roi = departamentos.filter(
    ee.Filter.And(
        ee.Filter.eq('ADM0_NAME', 'Colombia'),
        ee.Filter.eq('ADM1_NAME', 'Huila')
    )
).first().geometry()

# 2. Función para cargar Landsat 8 SR con índices y topografía
def cargar_landsat(fecha_inicio, fecha_fin):
    srtm = ee.Image("USGS/SRTMGL1_003")
    elev = srtm.select('elevation')
    slope = ee.Terrain.slope(elev)

    def procesar(img):
        img = img.select(
            ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'],
            ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']
        ).multiply(0.0000275).add(-0.2)

        ndvi = img.normalizedDifference(['NIR', 'Red']).rename('NDVI')
        ndwi = img.normalizedDifference(['Green', 'NIR']).rename('NDWI')
        ndsi = img.normalizedDifference(['Green', 'SWIR1']).rename('NDSI')

        return img.addBands([ndvi, ndwi, ndsi, elev.rename('Elevation'), slope.rename('Slope')]).clip(roi)

    return (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
            .filterBounds(roi)
            .filterDate(fecha_inicio, fecha_fin)
            .filter(ee.Filter.lt('CLOUD_COVER', 20))
            .map(procesar)
            .median())

# 3. Cargar imágenes
img2020 = cargar_landsat('2020-01-01', '2020-12-31')
img2024 = cargar_landsat('2024-01-01', '2024-03-31')

# 4. Cargar WorldCover 2020
worldcover = ee.ImageCollection('ESA/WorldCover/v100').first().clip(roi)

# 5. Muestreo estratificado balanceado
bandas = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'NDVI', 'NDWI', 'NDSI', 'Elevation', 'Slope']
samples = worldcover.addBands(img2020).stratifiedSample(
    numPoints=700,  # Puedes ajustar esto
    classBand='Map',
    region=roi,
    scale=30,
    seed=42,
    geometries=True
)

# 6. Convertir a DataFrame usando getInfo()
samples_info = samples.getInfo()
features = samples_info['features']

data = []
for feature in features:
    properties = feature['properties']
    data.append(properties)

df = pd.DataFrame(data)

X = df[bandas]
y = df['Map']

# 7. Entrenar modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
clf.fit(X_train, y_train)

# 8. Evaluar modelo
y_pred = clf.predict(X_test)
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))
print("Precisión:", accuracy_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

# 9. Guardar modelo localmente (opcional)
joblib.dump(clf, 'modelo_rf_huila.pkl')

# 10. Clasificación en GEE
trained_classifier = ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=3,
    minLeafPopulation=5,
    seed=42
).train(
    features=samples,
    classProperty='Map',
    inputProperties=bandas
)

clasificado_2020 = img2020.classify(trained_classifier).rename('clasificacion_2020')
clasificado_2024 = img2024.classify(trained_classifier).rename('clasificacion_2024')

# 11. Exportar clasificado 2024 a Google Drive
geemap.ee_export_image_to_drive(
    image=clasificado_2024,
    description='Huila_2024_Classified',
    folder='GEE_Exports',
    region=roi,
    scale=30,
    fileFormat='GeoTIFF',
    maxPixels=1e13
)

print("Exportación iniciada: Huila_2024_Classified → Google Drive / GEE_Exports")


Matriz de Confusión:
 [[149  19  29   3   2   8   0   0]
 [  9 154  29   5   4   9   0   0]
 [ 15  58  86  19   7  24   1   0]
 [  2   7   6 166  18   9   0   2]
 [ 10   7   7  29 145  12   0   0]
 [ 18   9  21  19  26 106  11   0]
 [  0   0   0   0   2  12 197   0]
 [  0   1   0   1   0   2   1   6]]
Precisión: 0.6808367071524967
Reporte de clasificación:
               precision    recall  f1-score   support

          10       0.73      0.71      0.72       210
          20       0.60      0.73      0.66       210
          30       0.48      0.41      0.44       210
          40       0.69      0.79      0.73       210
          50       0.71      0.69      0.70       210
          60       0.58      0.50      0.54       210
          80       0.94      0.93      0.94       211
          90       0.75      0.55      0.63        11

    accuracy                           0.68      1482
   macro avg       0.69      0.66      0.67      1482
weighted avg       0.68      0.68      0.68 